In [2]:
import pandas as pd
import numpy as np
import os
import tokenize_uk
from typing import List
import langdetect

# Data

In this task we selected computers and notebooks categories in rozetka.com. So we scribed the review comments data with mentined star for the next sites:

+ https://hard.rozetka.com.ua/ua/computers/c80095/ 
+ https://hard.rozetka.com.ua/ua/computers/c80095/
+ https://hard.rozetka.com.ua/

In [4]:
data_dir = "data"

In [5]:
! ls data

comments_urls.txt  rozetka-hard-comments_all.csv
hard_urls.txt	   rozetka-notebooks-comments_all.csv
pc_urls.txt	   rozetka-pc-comments_all.csv


In [102]:
def read_csv_from_dir(directory):
    frames = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{directory}/{filename}")
            frames.append(df)
            
    result = pd.concat(frames)
    return result

In [122]:
df = read_csv_from_dir(data_dir)

In [7]:
df = pd.read_csv("data/rozetka-hard-comments_all.csv")

In [8]:
len(df)

14009

In [9]:
df = df.fillna("")

In [11]:
df.head()

,Unnamed: 0,title,pros,cons,rating
0,0,Думаю на 2020 це ідеальний вибір. За ціну 1600...,Ціна - продуктивність,"Боксовий кулер краще відразу замінити , в прос...",5
1,1,Очень доволен покупкой. А тем более учитывая ч...,В первую очередь это его цена. На данный момен...,Работает как часы. Недостатков не обнаружил. Е...,5
2,2,Конкретно эту версию YD1600BBAFBOX не так давн...,Цена - качество,,5
3,3,"после fx6300, 1600af ракета.",,,5
4,4,"Норм проц, взял к нему в450, работает стабильн...","Охлаждение, производительность",За 80$ их просто нет,5


In [12]:
def is_suitable_text(text: str):
    if len(text) == 0:
        return True
    try:
        if langdetect.detect(text) == 'uk':
            return True
    except:
        return False
    return False

In [13]:
def filter_non_ukrainian_language(df):
    title_list = df['title'].values
    pros_list = df['pros'].values
    cons_list = df['cons'].values
    rate_list = df['rating'].values
    
    result = {
    'title': [],
    'pros' : [],
    'cons' : [],
    'rating' : []
}
    
    for i in range(0, len(title_list)):
        title = title_list[i].strip()
        pros = pros_list[i].strip()
        cons = cons_list[i].strip()
        
        if len(title) == 0 and len(pros) == 0 and len(cons) == 0:
            continue
        
        if is_suitable_text(title) is False:
            continue
        
        
       
        if is_suitable_text(pros) is False:
            continue

        
        if is_suitable_text(cons) is False:
            continue

        result['title'].append(title)
        result['pros'].append(pros)
        result['cons'].append(cons)
        result['rating'].append(rate_list[i])
        
    return pd.DataFrame(result)

In [14]:
def concatenate_title_pros_cons(df):
    title_list = df['title'].values
    pros_list = df['pros'].values
    cons_list = df['cons'].values
    y = df['rating'].values
    
    X = []
    for i in range(0, len(title_list)):
        text = f"{title_list[i]} {pros_list[i]} {cons_list[i]}"
        X.append(text)
        
    return X, y

In [15]:
def reduce_star_labels(y):
    reduced_y = []
    for yi in y:
        if yi >= 4:
            reduced_y.append('pos')
        elif yi == 3:
            reduced_y.append('neutral')
        else:
            reduced_y.append('neg')
    return reduced_y

In [16]:
uk_df = filter_non_ukrainian_language(df)

In [17]:
uk_df

,title,pros,cons,rating
0,Думаю на 2020 це ідеальний вибір. За ціну 1600...,Ціна - продуктивність,"Боксовий кулер краще відразу замінити , в прос...",5
1,Найкращий процесор 2019 по відношенню ціна/про...,"Ціна, 6 ядер.","Тільки для материнських плат 300 серії, необхі...",5
2,Придбав ССД на заміну основного жорсткого диск...,"Ціна, висока шкидкість запису та зчитування, с...",Наразі нема,5
3,На матплаті Asus TUF B450-Plus Gaming + Ryzen ...,3200 МГц при 16 таймінгах,,5
4,Питання до представника компанії. Чому не відп...,,,5
...,...,...,...,...
1426,"Давно займаюсь музикою, мав і маю, також багат...","За таку ціну в цьому діапазоні, мабуть одні з ...",При максимальному звуці в простої є невеличкий...,5
1427,Вогонь! Поставив другим на передню панель на в...,"Сильний повітряний потік, контроль оборотів, д...","На максималках шумний, але це швидше не недолі...",5
1428,Хороший вибір за не великі кошти. Працює на ура.,Якість та надійність.,,5
1429,,"все супер і комфортно, та радує юесбі перехідн...",,5


In [141]:
X_all, y_all = concatenate_title_pros_cons(uk_df)

In [142]:
y_all = reduce_star_labels(y_all)

In [143]:
classes = ['pos', 'neutral', 'neg']

In [145]:
X_all[:5]

['Думаю на 2020 це ідеальний вибір. За ціну 1600 отримуєте 2600 на 12 нм і новій архітектурі. Стоїть в Asus B450 tuf pro gaming. Ціна - продуктивність Боксовий кулер краще відразу замінити , в простої 40-45°',
 'Найкращий процесор 2019 по відношенню ціна/продуктивність, ігри літають, в зборці з RX 580 8G 60+ фпс. Процесор завівся з разу 10. Ціна, 6 ядер. Тільки для материнських плат 300 серії, необхідно оновлення біос якщо ревізія мат. плати 1.0',
 'Придбав ССД на заміну основного жорсткого диску. Перша перевага - повна тиша з корпусу. Також порадувала швидкість після завантаження накопичувача майже на 70% Зчитування 435 та Запис 516 МБ\\с.  Нижче прикріплюю тести: 1 - швидкість при підключенні по !!!САТА 2!!! 2 - швидкість при підключенні по САТА 3 3 - швидкість при підключенні по САТА 3 та заповненому ССД на 298 ГБ 4 - для порівняння мій інший ССД для ОС - Kingston HyperX Fury 120GB (2 роки 3 місяці в використанні)  Якщо комусь потрібно, то реальна доступна ємність 447 ГБ.  На даний 

In [146]:
sent_uk_df = pd.DataFrame({
    'text': X_all,
    'label': y_all
})

In [147]:
sent_uk_df.head()

,text,label
0,Думаю на 2020 це ідеальний вибір. За ціну 1600...,pos
1,Найкращий процесор 2019 по відношенню ціна/про...,pos
2,Придбав ССД на заміну основного жорсткого диск...,pos
3,На матплаті Asus TUF B450-Plus Gaming + Ryzen ...,pos
4,Питання до представника компанії. Чому не відп...,pos


## Data analysis

In [148]:
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [149]:
uk_df['rating'].iplot(
    kind='hist',
    bins=5,
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    title='Review rating dictribution')

In [157]:
sent_uk_df['label'].iplot(
    kind='hist',
    bins=3,
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    title='Sentiment label distribution')

As we can see from the diagrams above, the users mostly mentioned positive reviews than negative. <br/>
To avoid overfiting, while training model, we will select approximatelly the same ratio for each labels (it means if neutral contains 42 and negative contains 46 we will select 46 of positive reviews). 

## Clear data

## Split data

In [160]:
import random
from sklearn.model_selection import train_test_split

In [151]:
def split_for_avoiding_overfitting(X, y, epsilon_percent = 0.2):
    X_y = list(zip(X, y))
    label_Xy_dict = dict()
    for i in range(0, len(X_y)):
        label = X_y[i][1]
        
        if label not in label_Xy_dict:
            label_Xy_dict[label] = [X_y[i]]
        else:
            label_Xy_dict[label].append(X_y[i])
            
    labels = list(label_Xy_dict.keys())
    labels.sort(key = lambda label: len(label_Xy_dict[label]))
    
    prev_len = None
    for label in labels:
        cur_len = len(label_Xy_dict[label])
        if prev_len is None:
            prev_len = cur_len
            continue
        
        if 1 - prev_len / cur_len <= epsilon_percent:
            continue
        
        label_Xy_dict[label] = random.sample(label_Xy_dict[label], int((1 + epsilon_percent) * prev_len))
        
    
    new_X_y = []
    for label in label_Xy_dict:
        new_X_y += label_Xy_dict[label]
        
    random.shuffle(new_X_y)
    
    res = list(zip(*new_X_y)) 
    X_new = list(res[0])
    y_new = list(res[1])
    
    return X_new, y_new
    

In [152]:
X, y = split_for_avoiding_overfitting(X_all, y_all)

In [156]:
new_sent_uk_df = pd.DataFrame({
    'text': X,
    'label': y
})

In [158]:
new_sent_uk_df['label'].iplot(
    kind='hist',
    bins=3,
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    title='Sentiment label distribution')

As we can see from the diagram above, it has approximately uniform distribution to avoid overfitiing for one specific label (e.g. pos)

In [161]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bag of words

In [87]:
import tokenize_uk
from sklearn.feature_extraction.text import CountVectorizer

In [116]:
class BagOfWordsFeatureExtractor:
    NUM_FEATURES = 5000
    
    def __init__(self, corpus: List[str], stop_words: List[str] = None):
        self.corpus: list[str] = corpus
        self.vectorizer = CountVectorizer(analyzer = "word", 
                                          tokenizer = tokenize_uk.tokenize_words, 
                                          preprocessor = None, 
                                          stop_words = stop_words, 
                                          max_features = None) 
        self.train_data_features = self.vectorizer.fit_transform(self.corpus)
        
    
    def transform(self, text_list: List[str]):
        return self.vectorizer.transform(text_list).toarray()

In [120]:
bow_feature_extractor = BagOfWordsFeatureExtractor(X)

In [170]:
bow_feature_extractor.transform(["Думаю на 2020 це ідеальний вибір."])

array([[0, 0, 0, ..., 0, 0, 0]])

In [122]:
bow_feature_extractor.transform(["Думаю на 2020 це ідеальний вибір."])[0].sum()

7

In [182]:
x_train_features = bow_feature_extractor.transform(X_train)
x_test_features = bow_feature_extractor.transform(X_test)

# Naive Bayes Classifier

#### Gaussian Naive Bayes

In [179]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [167]:
gnb = GaussianNB()

In [174]:
gnb.fit(x_train_features, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [177]:
y_pred = gnb.predict(x_test_features)

In [180]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.38      0.43      0.40         7
     neutral       0.38      0.33      0.35         9
         pos       0.75      0.75      0.75        12

    accuracy                           0.54        28
   macro avg       0.50      0.50      0.50        28
weighted avg       0.54      0.54      0.53        28



#### Multinomial Naive Bayes

In [181]:
from sklearn.naive_bayes import MultinomialNB

In [184]:
multinomial_nb = MultinomialNB().fit(x_train_features, y_train)

In [185]:
y_pred = multinomial_nb.predict(x_test_features)

In [186]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.30      0.86      0.44         7
     neutral       0.00      0.00      0.00         9
         pos       0.62      0.42      0.50        12

    accuracy                           0.39        28
   macro avg       0.31      0.42      0.31        28
weighted avg       0.34      0.39      0.33        28



/home/dbabenko/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

